In [1]:
import random, os
import numpy as np
import torch
import glob

def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [3]:
import pandas as pd

test_df = pd.read_csv('train/cleaned_train.csv')
test_df['date_orig'] = pd.to_datetime(test_df['date'], format='%Y-%m-%d %H:%M:%S')
test_df.head()

,id,date,employee,action,date_orig
0,1256,2022-05-24 08:08:36,2,15.0,2022-05-24 08:08:36
1,1257,2022-05-24 08:08:37,2,4.0,2022-05-24 08:08:37
2,1258,2022-05-24 08:12:13,3,15.0,2022-05-24 08:12:13
3,1259,2022-05-24 08:12:25,3,2.0,2022-05-24 08:12:25
4,1260,2022-05-24 08:12:42,3,12.0,2022-05-24 08:12:42


In [4]:
SPLIT_TRAIN1_AND_TRAIN2_ID=2522
test_df=test_df[test_df['id']>=SPLIT_TRAIN1_AND_TRAIN2_ID]
test_df.head()

,id,date,employee,action,date_orig
1266,2522,2022-05-26 08:04:49,6,13.0,2022-05-26 08:04:49
1267,2523,2022-05-26 08:04:54,6,4.0,2022-05-26 08:04:54
1268,2524,2022-05-26 08:07:00,6,14.0,2022-05-26 08:07:00
1269,2525,2022-05-26 08:07:11,6,5.0,2022-05-26 08:07:11
1270,2526,2022-05-26 08:15:16,1,15.0,2022-05-26 08:15:16


In [6]:
parsed_time_test_df=pd.read_csv('parsed_time_train2.txt',sep='\t',names=['frame','date'])
parsed_time_test_df['date_orig'] = pd.to_datetime(parsed_time_test_df['date'], format='%m-%d-%Y %a %H:%M:%S', errors='coerce')
parsed_time_test_df = parsed_time_test_df.dropna(subset=['date_orig'])
parsed_time_test_df['date'] = parsed_time_test_df['date_orig'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
# parsed_time_test_df=parsed_time_test_df.set_index('frame')
parsed_time_test_df.head()

,frame,date,date_orig
0,1,2022-05-26 08:02:47,2022-05-26 08:02:47
1,2,2022-05-26 08:02:47,2022-05-26 08:02:47
2,3,2022-05-26 08:02:47,2022-05-26 08:02:47
3,4,2022-05-26 08:02:47,2022-05-26 08:02:47
4,5,2022-05-26 08:02:47,2022-05-26 08:02:47


In [7]:
df = pd.merge(test_df, parsed_time_test_df, on=['date'], how="inner", indicator=True)
# df = df[df['_merge'] == 'left_only']
df = df[df['_merge'] == 'both']
df.head()

,id,date,employee,action,date_orig_x,frame,date_orig_y,_merge
0,2522,2022-05-26 08:04:49,6,13.0,2022-05-26 08:04:49,282,2022-05-26 08:04:49,both
1,2522,2022-05-26 08:04:49,6,13.0,2022-05-26 08:04:49,283,2022-05-26 08:04:49,both
2,2522,2022-05-26 08:04:49,6,13.0,2022-05-26 08:04:49,284,2022-05-26 08:04:49,both
3,2522,2022-05-26 08:04:49,6,13.0,2022-05-26 08:04:49,285,2022-05-26 08:04:49,both
4,2522,2022-05-26 08:04:49,6,13.0,2022-05-26 08:04:49,286,2022-05-26 08:04:49,both


In [8]:
from catboost import CatBoostClassifier

door_model = CatBoostClassifier()
door_model.load_model("door_predict.cbm")

In [9]:
people_detect_model = torch.hub.load('ultralytics/yolov5','custom', 'yolov5x6') #'ultralytics/yolov5', 'custom', 'yolov5s-cls.pt'
people_detect_model.classes = [0]

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-8-10 Python-3.8.10 torch-1.13.0.dev20220701+cu116 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 


In [10]:
people_detect_model.cuda()
people_detect_model.eval()

AutoShape(
  (model): DetectMultiBackend(
    (model): Model(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 80, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(80, 80, kernel_size=(1, 1), stride=(1, 1))
      

In [11]:
test_df.shape

(2253, 5)

In [12]:
from torch import nn
import torch.nn.functional as F
import timm # PyTorch Image Models

model = timm.create_model('tf_efficientnet_b4_ns',pretrained=True) #load pretrained model
for param in model.parameters():
    param.requires_grad=False

#orginally, it was:
#(classifier): Linear(in_features=1792, out_features=1000, bias=True)


#we are updating it as a 2-class classifier:
model.classifier = nn.Sequential(
    nn.Linear(in_features=1792, out_features=625), #1792 is the orginal in_features
    nn.ReLU(), #ReLu to be the activation function
    nn.Dropout(p=0.3),
    nn.Linear(in_features=625, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=10), 
)
model.to(device)
model.load_state_dict(torch.load('EmplDetectModel.pt'))
model.eval()

/usr/local/lib/python3.8/dist-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)
Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b4_ns-d6313a46.pth)


EfficientNet(
  (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          24, ep

In [13]:
from torchvision import transforms as T

train_transform = T.Compose([
     T.Resize(size=(224,224)), # Resizing the image to be 224 by 224
     T.ToTensor(), #converting the dimension from (height,weight,channel) to (channel,height,weight) convention of PyTorch
     T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]) # Normalize by 3 means 3 StD's of the image net, 3 channels

])

In [16]:
employee_class_name = ['0','1','10','11','2','3','4','5','6','8']

In [17]:
test_df.head()

,id,date,employee,action,date_orig
1266,2522,2022-05-26 08:04:49,6,13.0,2022-05-26 08:04:49
1267,2523,2022-05-26 08:04:54,6,4.0,2022-05-26 08:04:54
1268,2524,2022-05-26 08:07:00,6,14.0,2022-05-26 08:07:00
1269,2525,2022-05-26 08:07:11,6,5.0,2022-05-26 08:07:11
1270,2526,2022-05-26 08:15:16,1,15.0,2022-05-26 08:15:16


In [18]:
import cv2
from PIL import Image

test_path="/home/hacaton_irkutsk/train/train2"
door_coord_items=[]
for ind in test_df.index:
    print(ind)
    row_id=int(test_df['id'][ind])
    row_frames_df=df[df['id']==row_id]
    
    uniq_frames=set()
    for fr_ind in row_frames_df.index:
        frame=int(row_frames_df['frame'][fr_ind])
        uniq_frames.add(frame)
        
    if len(uniq_frames)==0:
        continue
        
    min_frame=min(uniq_frames)
    max_frame=max(uniq_frames)
    ct=parsed_time_test_df['date_orig'][min_frame]
    
    for i in range(min_frame-10,max_frame+10):
        cframe=int(parsed_time_test_df['frame'][i])
        dt=parsed_time_test_df['date_orig'][i]
        if ct<dt:
            delta=dt-ct
        else:
            delta=ct-dt
        if delta.total_seconds()<4:
            uniq_frames.add(cframe)
        
    front_door_borders=[300,100,1050,1350]
    back_door_borders=[100,1000,400,1400]
    crop_boxes=[front_door_borders,back_door_borders]
    
    
    print('uniq_frames=',uniq_frames)
    for i in uniq_frames:
        img = cv2.imread(f"{test_path}/out{int(i)}.png", cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        newsize = (1920,1080)
        resized = cv2.resize(img, newsize, interpolation = cv2.INTER_AREA)
        
        for crop_box in crop_boxes:
            # front doors
            crop = resized[crop_box[0]:crop_box[2],crop_box[1]:crop_box[3]]

            results = people_detect_model(crop)
            if results.xyxy[0].shape == torch.Size([1, 6]):
#                 print('detect')
                find_region=results.xyxy[0][0].cpu().numpy()
                bbox=np.array(find_region[:4])
                bbox = bbox.astype(int)
                bbox[1]+=crop_box[0]
                bbox[3]+=crop_box[0]
                bbox[0]+=crop_box[1]
                bbox[2]+=crop_box[1]
                
                employee=0
                
                delta_y=0
                delta_x=0
                delta=(bbox[3]-bbox[1])-(bbox[2]-bbox[0])
                if delta>0:
                    delta_x=int(delta/2)
                else:
                    delta_y=int(delta/2)

                crop=resized[bbox[1]-delta_y:bbox[3]+delta_y, bbox[0]-delta_x:bbox[2]+delta_x]
#                 print(crop.shape)
                if crop.shape[0]!=0 and crop.shape[1]!=0:
                    im = Image.fromarray(crop)
                    resized_crop = train_transform(im)

    #                 print('resized_crop.shape=',resized_crop.shape)
                    ps = model(resized_crop.to(device).unsqueeze(0))
                    ps = F.softmax(ps,dim = 1)
                    ps = ps.cpu().data.numpy().squeeze()
                    employee_index=np.argmax(ps, axis=0)
                    employee=employee_class_name[employee_index]
                
                door_coord_items.append({
                    'row_id': row_id,
                    'frame_id':i,
                    'center_frame': min_frame,
                    'employee':employee,
                    'xmin':bbox[0],
                    'ymin':bbox[1],
                    'xmax':bbox[2],
                    'ymax':bbox[3],
                })
    


1266
uniq_frames= {273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296}
1267
uniq_frames= {298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321}
1268
uniq_frames= {384, 385, 386, 387, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383}
1269
uniq_frames= {419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439}
1270
uniq_frames= {440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463}
1271
uniq_frames= {450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473}
1272
uniq_frames= {553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576}
1273
uniq_frames= {558, 559, 560, 561, 562

1322
uniq_frames= {3006, 3007, 3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017, 3018, 3019, 3020, 3021, 3022, 3023, 3024, 3025, 3026, 3027, 3028, 3029}
1323
uniq_frames= {3011, 3012, 3013, 3014, 3015, 3016, 3017, 3018, 3019, 3020, 3021, 3022, 3023, 3024, 3025, 3026, 3027, 3028, 3029, 3030, 3031, 3032, 3033, 3034}
1324
uniq_frames= {3031, 3032, 3033, 3034, 3035, 3036, 3037, 3038, 3039, 3040, 3041, 3042, 3043, 3044, 3045, 3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054}
1325
uniq_frames= {3036, 3037, 3038, 3039, 3040, 3041, 3042, 3043, 3044, 3045, 3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054, 3055, 3056, 3057, 3058, 3059}
1326
uniq_frames= {3084, 3085, 3086, 3087, 3088, 3089, 3090, 3091, 3092, 3093, 3094, 3095, 3096, 3097, 3098, 3099, 3100, 3101, 3102, 3103, 3104, 3105, 3106}
1327
uniq_frames= {3088, 3089, 3090, 3091, 3092, 3093, 3094, 3095, 3096, 3097, 3098, 3099, 3100, 3101, 3102, 3103, 3104, 3105, 3106, 3107, 3108, 3109, 3110, 3111}
1328
uniq_frames= {3088, 308

1374
uniq_frames= {5582, 5583, 5584, 5585, 5586, 5587, 5588, 5589, 5590, 5591, 5592, 5593, 5594, 5595, 5596, 5597, 5598, 5599, 5600, 5601, 5602, 5603, 5604, 5605}
1375
uniq_frames= {5637, 5638, 5639, 5640, 5641, 5642, 5643, 5644, 5645, 5646, 5647, 5648, 5649, 5650, 5651, 5652, 5653, 5654, 5655, 5656, 5657, 5658, 5659, 5660}
1376
uniq_frames= {5647, 5648, 5649, 5650, 5651, 5652, 5653, 5654, 5655, 5656, 5657, 5658, 5659, 5660, 5661, 5662, 5663, 5664, 5665, 5666, 5667, 5668, 5669, 5670}
1377
uniq_frames= {5802, 5803, 5804, 5805, 5806, 5807, 5808, 5809, 5810, 5811, 5812, 5813, 5814, 5815, 5816, 5817, 5818, 5819, 5820, 5821, 5822, 5823, 5824}
1378
uniq_frames= {5811, 5812, 5813, 5814, 5815, 5816, 5817, 5818, 5819, 5820, 5821, 5822, 5823, 5824, 5825, 5826, 5827, 5828, 5829, 5830, 5831, 5832, 5833, 5834}
1379
uniq_frames= {5858, 5859, 5860, 5861, 5862, 5863, 5864, 5865, 5866, 5867, 5868, 5869, 5870, 5871, 5872, 5873, 5874, 5875, 5876, 5877, 5878, 5879, 5880, 5881}
1380
uniq_frames= {5898, 589

1426
uniq_frames= {10402, 10403, 10404, 10405, 10406, 10407, 10408, 10409, 10410, 10411, 10412, 10413, 10414, 10415, 10416, 10417, 10418, 10419, 10420, 10421, 10422, 10423, 10424, 10425}
1427
uniq_frames= {10457, 10458, 10459, 10460, 10461, 10462, 10463, 10464, 10465, 10466, 10467, 10468, 10469, 10470, 10471, 10472, 10473, 10474, 10475, 10476, 10477, 10478, 10479, 10480}
1428
uniq_frames= {10550, 10551, 10552, 10553, 10554, 10555, 10556, 10557, 10558, 10559, 10560, 10561, 10562, 10563, 10564, 10565, 10566, 10567, 10568, 10569, 10570, 10571, 10572, 10573}
1429
uniq_frames= {10555, 10556, 10557, 10558, 10559, 10560, 10561, 10562, 10563, 10564, 10565, 10566, 10567, 10568, 10569, 10570, 10571, 10572, 10573, 10574, 10575, 10576, 10577, 10578}
1430
uniq_frames= {10595, 10596, 10597, 10598, 10599, 10600, 10601, 10602, 10603, 10604, 10605, 10606, 10607, 10608, 10609, 10610, 10611, 10612, 10613, 10614, 10615, 10616, 10617, 10618}
1431
uniq_frames= {10633, 10634, 10635, 10636, 10637, 10638, 1063

1472
uniq_frames= {12633, 12634, 12635, 12636, 12637, 12638, 12639, 12640, 12641, 12642, 12643, 12644, 12645, 12646, 12647, 12648, 12649, 12650, 12651, 12652, 12653, 12654, 12655, 12656}
1473
uniq_frames= {12648, 12649, 12650, 12651, 12652, 12653, 12654, 12655, 12656, 12657, 12658, 12659, 12660, 12661, 12662, 12663, 12664, 12665, 12666, 12667, 12668, 12669, 12670, 12671}
1474
uniq_frames= {12696, 12697, 12698, 12699, 12700, 12701, 12702, 12703, 12704, 12705, 12706, 12707, 12708, 12709, 12710, 12711, 12712, 12713, 12714, 12715, 12716, 12717, 12718, 12719}
1475
uniq_frames= {12800, 12801, 12802, 12803, 12804, 12805, 12806, 12807, 12808, 12809, 12810, 12811, 12812, 12813, 12814, 12815, 12816, 12817, 12798, 12799}
1476
uniq_frames= {12804, 12805, 12806, 12807, 12808, 12809, 12810, 12811, 12812, 12813, 12814, 12815, 12816, 12817, 12818, 12819, 12820, 12821, 12822, 12823, 12824, 12825, 12826, 12827}
1477
uniq_frames= {12853, 12854, 12855, 12856, 12857, 12858, 12859, 12860, 12861, 12862, 1286

...

3309
uniq_frames= {88291, 88292, 88293, 88294, 88295, 88296, 88297, 88298, 88299, 88300, 88301, 88302, 88303, 88304, 88305, 88306, 88307, 88308, 88309, 88310, 88311, 88312, 88313, 88314}
3310
uniq_frames= {88296, 88297, 88298, 88299, 88300, 88301, 88302, 88303, 88304, 88305, 88306, 88307, 88308, 88309, 88310, 88311, 88312, 88313, 88314, 88315, 88316, 88317, 88318, 88319}
3311
uniq_frames= {88320, 88321, 88322, 88323, 88324, 88325, 88326, 88327, 88328, 88329, 88306, 88307, 88308, 88309, 88310, 88311, 88312, 88313, 88314, 88315, 88316, 88317, 88318, 88319}
3312
uniq_frames= {88326, 88327, 88328, 88329, 88330, 88331, 88332, 88333, 88334, 88335, 88336, 88337, 88338, 88339, 88340, 88341, 88342, 88343, 88344, 88345, 88346, 88347, 88348, 88349}
3313
uniq_frames= {88376, 88377, 88378, 88379, 88380, 88381, 88382, 88383, 88384, 88385, 88386, 88387, 88388, 88389, 88390, 88391, 88392, 88393, 88394, 88395, 88396, 88397, 88398, 88399}
3314
uniq_frames= {88451, 88452, 88453, 88454, 88455, 88456, 8845

3353
uniq_frames= {89900, 89901, 89902, 89903, 89904, 89905, 89906, 89907, 89908, 89909, 89910, 89911, 89912, 89913, 89914, 89915, 89916, 89917, 89918, 89919, 89920, 89921, 89922, 89923}
3354
uniq_frames= {89984, 89985, 89986, 89987, 89988, 89989, 89990, 89991, 89992, 89993, 89994, 89995, 89996, 89997, 89998, 89975, 89976, 89977, 89978, 89979, 89980, 89981, 89982, 89983}
3355
uniq_frames= {90010, 90011, 90012, 90013, 90014, 90015, 90016, 90017, 90018, 90019, 90020, 90021, 90022, 90023, 90024, 90025, 90026, 90027, 90028, 90029, 90030, 90031, 90032, 90033}
3356
uniq_frames= {90035, 90036, 90037, 90038, 90039, 90040, 90041, 90042, 90043, 90044, 90045, 90046, 90047, 90048, 90049, 90050, 90051, 90052, 90053, 90054, 90055, 90056, 90057, 90058}
3357
uniq_frames= {90085, 90086, 90087, 90088, 90089, 90090, 90091, 90092, 90093, 90094, 90095, 90096, 90097, 90098, 90099, 90100, 90101, 90102, 90103, 90104, 90105, 90106, 90107, 90108}
3358
uniq_frames= {90240, 90241, 90242, 90243, 90244, 90245, 9024

3398
uniq_frames= {92735, 92736, 92737, 92738, 92739, 92740, 92741, 92742, 92743, 92744, 92745, 92746, 92747, 92748, 92749, 92750, 92751, 92752, 92753, 92754, 92755, 92756, 92757, 92758}
3399
uniq_frames= {92845, 92846, 92847, 92848, 92849, 92850, 92851, 92852, 92853, 92854, 92855, 92856, 92857, 92858, 92859, 92860, 92861, 92862, 92863, 92864, 92865, 92866, 92867, 92868}
3400
uniq_frames= {93124, 93125, 93126, 93127, 93128, 93129, 93130, 93131, 93132, 93133, 93134, 93135, 93136, 93137, 93138, 93139, 93140, 93141, 93142, 93143, 93144, 93145, 93146, 93147, 93148, 93149}
3401
uniq_frames= {93269, 93270, 93271, 93272, 93273, 93274, 93275, 93276, 93277, 93278, 93279, 93280, 93281, 93282, 93283, 93284, 93285, 93286, 93287, 93288, 93289, 93290, 93291, 93292, 93293, 93294}
3402
uniq_frames= {93524, 93525, 93526, 93527, 93528, 93529, 93530, 93531, 93532, 93533, 93534, 93535, 93536, 93537, 93538, 93539, 93540, 93541, 93542, 93543, 93544, 93545, 93546, 93547, 93548, 93549}
3403
uniq_frames= {9362

3441
uniq_frames= {96256, 96257, 96258, 96259, 96260, 96261, 96262, 96263, 96264, 96265, 96266, 96242, 96243, 96244, 96245, 96246, 96247, 96248, 96249, 96250, 96251, 96252, 96253, 96254, 96255}
3442
uniq_frames= {96256, 96257, 96258, 96259, 96260, 96261, 96262, 96263, 96264, 96265, 96266, 96267, 96268, 96269, 96270, 96271, 96272, 96273, 96274, 96275, 96276, 96277, 96278, 96279, 96280, 96281}
3443
uniq_frames= {96331, 96332, 96333, 96334, 96335, 96336, 96337, 96338, 96339, 96340, 96341, 96342, 96343, 96344, 96345, 96346, 96347, 96348, 96349, 96350, 96351, 96352, 96353, 96354, 96355, 96356}
3444
uniq_frames= {96417, 96418, 96419, 96420, 96421, 96422, 96423, 96424, 96425, 96426, 96427, 96428, 96429, 96430, 96431, 96432, 96433, 96434, 96435, 96436, 96437, 96438, 96439, 96440, 96441, 96442}
3445
uniq_frames= {96467, 96468, 96469, 96470, 96471, 96472, 96473, 96474, 96475, 96476, 96477, 96478, 96479, 96480, 96481, 96482, 96483, 96484, 96485, 96486, 96487, 96488, 96489, 96490, 96491, 96492}
34

3482
uniq_frames= {98889, 98890, 98891, 98892, 98893, 98894, 98895, 98896, 98897, 98898, 98899, 98900, 98901, 98902, 98903, 98904, 98905, 98906, 98907, 98908, 98909, 98910, 98911, 98912, 98913, 98914}
3483
uniq_frames= {98944, 98945, 98946, 98947, 98948, 98949, 98924, 98925, 98926, 98927, 98928, 98929, 98930, 98931, 98932, 98933, 98934, 98935, 98936, 98937, 98938, 98939, 98940, 98941, 98942, 98943}
3484
uniq_frames= {98966, 98967, 98968, 98969, 98970, 98971, 98972, 98973, 98974, 98975, 98976, 98977, 98978, 98979, 98980, 98981, 98982, 98983, 98984, 98985, 98986, 98987, 98988, 98989, 98990, 98991}
3485
uniq_frames= {98996, 98997, 98998, 98999, 99000, 99001, 99002, 99003, 99004, 99005, 99006, 99007, 99008, 99009, 99010, 99011, 99012, 99013, 99014, 99015, 99016, 99017, 99018, 99019, 99020, 99021}
3486
uniq_frames= {99036, 99037, 99038, 99039, 99040, 99041, 99042, 99043, 99044, 99045, 99046, 99047, 99048, 99049, 99050, 99051, 99052, 99053, 99054, 99055, 99056, 99057, 99058, 99059, 99060, 99

In [19]:
len(door_coord_items)

29207

In [20]:
door_coord_df = pd.DataFrame(door_coord_items)
door_coord_df.head(20)

,row_id,frame_id,center_frame,employee,xmin,ymin,xmax,ymax
0,2522,278,282,6,1156,300,1191,405
1,2522,279,282,0,1157,300,1192,406
2,2522,280,282,0,1157,301,1199,407
3,2522,281,282,0,1158,300,1199,407
4,2522,282,282,0,1157,300,1200,407
5,2522,283,282,0,1158,300,1202,406
6,2522,284,282,0,1160,300,1202,403
7,2522,285,282,0,1159,300,1203,403
8,2522,286,282,3,1169,287,1209,398
9,2522,287,282,0,1172,300,1219,399


In [21]:
door_coord_df.shape

(29207, 8)

In [22]:
pred = door_model.predict(door_coord_df[['xmin','ymin','xmax','ymax']])
door_coord_df['pred']=pred
door_coord_df.head()

,row_id,frame_id,center_frame,employee,xmin,ymin,xmax,ymax,pred
0,2522,278,282,6,1156,300,1191,405,2
1,2522,279,282,0,1157,300,1192,406,2
2,2522,280,282,0,1157,301,1199,407,2
3,2522,281,282,0,1158,300,1199,407,2
4,2522,282,282,0,1157,300,1200,407,2


In [23]:
# door_coord_df[door_coord_df['row_id']==245].head(40)

In [24]:

door_coord_df[['employee']] = door_coord_df[['employee']].astype(int)

In [ ]:
# door_coord_df['x_c']=(door_coord_df['x_c'])

In [71]:
result_actions={}
result_employee={}

new_result_actions={}


In [67]:
#2523
df_2523 = door_coord_df[door_coord_df['row_id']==2523]
df_2523.head()

,row_id,frame_id,center_frame,employee,xmin,ymin,xmax,ymax,pred
29,2523,298,307,3,1222,300,1257,377,3
30,2523,298,307,0,1222,278,1257,379,3
31,2523,299,307,3,1225,300,1261,374,3
32,2523,299,307,3,1225,278,1260,375,4
33,2523,300,307,10,1225,300,1263,375,2


In [119]:
df_3550=door_coord_df[door_coord_df['row_id']==3550]
df_3550.head(20)

,row_id,frame_id,center_frame,employee,xmin,ymin,xmax,ymax,pred
13614,3550,53006,53004,0,1135,523,1266,698,9
13615,3550,53008,53004,0,1053,511,1180,833,8
13616,3550,53009,53004,0,1023,504,1148,834,8
13617,3550,53010,53004,3,984,513,1121,832,8
13618,3550,53011,53004,0,976,478,1117,816,2
13619,3550,53012,53004,0,978,461,1081,790,2
13620,3550,53013,53004,0,979,460,1077,789,2
13621,3550,53014,53004,0,990,449,1080,751,2
13622,3550,53015,53004,0,998,440,1086,728,2
13623,3550,53016,53004,0,1012,440,1095,723,2


In [87]:
def fine_tuning_4_5_doors(actions_df, cv):
    if cv not in [4,5,14,15]:
        return cv
    
    
    less_count=0
    used_frames=set()
    closest_frame_coord=-1
    closest_frame_index=10000
    for ind in actions_df.index:
        frame=actions_df['frame_id'][ind]
        center_frame=actions_df['center_frame'][ind]
        xmin=actions_df['xmin'][ind]
        xmax=actions_df['xmax'][ind]
        
        
        if abs(center_frame-frame)<closest_frame_index:
            closest_frame_index=abs(center_frame-frame)
            closest_frame_coord=int((xmax+xmin)/2)
        if frame in used_frames:
            continue
        used_frames.add(frame)
        if frame<center_frame:
            less_count+=1
            
#         print(ind)
    to_door=less_count>2 #cv in [4,5]
    if to_door:
        if closest_frame_coord<1270:
            return 4
        else:
            return 5
    else:
        if closest_frame_coord<1270:
            return 14
        else:
            return 15
    
    return cv
 
fine_tuning_4_5_doors(df_2523,14)

4

In [122]:
def fine_tuning_18_12_doors(actions_df, cv):
    if cv not in [12]:
        return cv
    
    
    less_count=0
    used_frames=set()
    closest_frame_coord=-1
    closest_frame_index=10000
    for ind in actions_df.index:
        frame=actions_df['frame_id'][ind]
        center_frame=actions_df['center_frame'][ind]
        xmin=actions_df['xmin'][ind]
        xmax=actions_df['xmax'][ind]
        
        
        if abs(center_frame-frame)<closest_frame_index:
            closest_frame_index=abs(center_frame-frame)
            closest_frame_coord=int((xmax+xmin)/2)
        if frame in used_frames:
            continue
        used_frames.add(frame)
        if frame<center_frame:
            less_count+=1
    
    if closest_frame_coord<1000:
        return 12
    else:
        return 18
    
    return cv
 
fine_tuning_18_12_doors(door_coord_df[door_coord_df['row_id']==3677],12)

18

In [123]:

for ind in test_df.index:
    row_id=int(test_df['id'][ind])
    actions_df=door_coord_df[door_coord_df['row_id']==row_id]
    actions_df = actions_df.sort_values(by ='frame_id' )
    if actions_df.shape[0]==0:
        continue
    
    frame_preds={}
    for i in actions_df.index:
        center_frame=actions_df['center_frame'][i] 
#         print(center_frame)
        frame_id=actions_df['frame_id'][i]
        
        door=actions_df['pred'][i]
        frame_preds[frame_id]=door
#         print(center_frame,frame_id,door)
#     print(center_frame)

    prev_count={}
    next_count={}
    row_frames=frame_preds.keys()
    sorted(row_frames, reverse=True)
    for f in row_frames:
        if f>center_frame:
            continue
            
        if frame_preds[f] not in prev_count:
            prev_count[frame_preds[f]]=0
        prev_count[frame_preds[f]]+=1
    sorted(row_frames, reverse=False)
    for f in row_frames:
        if f<=center_frame:
            continue
            
        if frame_preds[f] not in next_count:
            next_count[frame_preds[f]]=0
        next_count[frame_preds[f]]+=1
        
    prev_max_val=-1
    prev_max_val_door=-1
    for k in prev_count:
        if prev_count[k]>prev_max_val:
            prev_max_val=prev_count[k]
            prev_max_val_door=k
            
    next_max_val=-1
    next_max_val_door=-1
    for k in next_count:
        if next_count[k]>next_max_val:
            next_max_val=next_count[k]
            next_max_val_door=k
    
    if prev_max_val>=next_max_val:
        empl_df=actions_df[actions_df['frame_id']<=center_frame].tail(5)
        result_actions[row_id]=prev_max_val_door
    else:
        empl_df=actions_df[actions_df['frame_id']>=center_frame].head(5)
        result_actions[row_id]=next_max_val_door+10
    
    new_result_actions[row_id] = fine_tuning_4_5_doors(actions_df,result_actions[row_id])
    new_result_actions[row_id] = fine_tuning_18_12_doors(actions_df,result_actions[row_id])
        
    values, counts = np.unique(empl_df['employee'], return_counts=True)
    freq_ind = np.argmax(counts)

    result_employee[row_id]=values[freq_ind]
    
        
        
len(result_actions)
    
            
            

1999

In [33]:
test_df.head()

,id,date,employee,action,date_orig
1266,2522,2022-05-26 08:04:49,6,13.0,2022-05-26 08:04:49
1267,2523,2022-05-26 08:04:54,6,4.0,2022-05-26 08:04:54
1268,2524,2022-05-26 08:07:00,6,14.0,2022-05-26 08:07:00
1269,2525,2022-05-26 08:07:11,6,5.0,2022-05-26 08:07:11
1270,2526,2022-05-26 08:15:16,1,15.0,2022-05-26 08:15:16


In [124]:
action_stat_correct=0
empl_stat_correct=0
new_bad_actions={}
bad_actions={}
bad_empls={}
for ind in test_df.index:
    id=int(test_df['id'][ind])
    act=int(test_df['action'][ind])
    empl=int(test_df['employee'][ind])
    if id in result_actions:
        if act==result_actions[id]:
            action_stat_correct+=1
        elif act in [18]:
            print(id,result_actions[id])
        
        
        if act not in bad_actions:
            bad_actions[act]=[]
        bad_actions[act].append(result_actions[id])
        
        if act not in new_bad_actions:
            new_bad_actions[act]=[]
        new_bad_actions[act].append(new_result_actions[id])
    
    
    if id in result_employee:
        if empl==result_employee[id]:
            empl_stat_correct+=1
#         if act in [4,14,5,15]:
#             print(id)
        if empl not in bad_empls:
            bad_empls[empl]=[]
        bad_empls[empl].append(result_employee[id])
            
print('Empl=',empl_stat_correct,' of ',test_df.shape[0])
print('Action=',action_stat_correct,' of ',test_df.shape[0])
       

3550 12
3556 12
3558 19
3628 19
3668 19
3673 19
3675 8
3677 12
3681 9
3685 19
3686 9
3688 12
3692 2
3693 2
3696 19
3699 15
3701 15
3704 9
3705 2
3728 2
3729 12
3733 13
3739 19
3751 12
3756 2
3763 19
3764 12
3769 12
3786 12
3788 19
3794 12
3795 13
3797 3
3798 13
3799 13
3803 14
3807 19
3808 11
3811 19
3815 19
3816 12
3817 12
3821 19
3823 12
3826 8
3834 19
3843 9
3846 12
3847 12
3850 12
3855 14
3856 14
3869 19
3871 12
3872 13
3879 12
3881 12
3883 12
3895 12
3902 15
3905 12
3917 2
3927 12
3932 14
3935 12
3936 12
3942 14
3946 12
3948 12
3949 15
3952 12
3961 12
3962 12
3969 12
3971 12
3975 12
3976 12
3981 12
3989 12
3999 12
4003 12
4013 12
4014 12
4015 12
4029 12
4031 12
4041 12
4042 13
4045 12
4047 12
4049 12
4051 12
4057 12
4065 12
4066 12
4069 12
4071 12
4073 12
4079 12
4085 12
4093 12
4107 12
4121 12
4127 12
4130 12
4143 12
4145 12
4147 12
4151 12
4157 12
4169 12
4173 12
4181 19
4187 15
4197 12
4207 12
4223 12
4229 12
4237 12
4238 12
4241 14
4243 15
4245 15
4267 12
4269 12
4271 12
4278 

In [126]:
for k in bad_actions:
    print('action ',k)
    x = np.array(bad_actions[k])
    unique, counts = np.unique(x, return_counts=True)
    for i in range(len(unique)):
        print(f'{unique[i]}: {counts[i]}')
    print('****************')
    

action  13
4: 3
5: 1
12: 20
13: 41
14: 7
15: 5
16: 2
****************
action  4
2: 8
4: 85
5: 41
12: 7
13: 5
14: 38
15: 13
19: 1
****************
action  14
2: 6
3: 3
4: 22
5: 4
12: 20
13: 7
14: 109
15: 62
16: 1
19: 1
****************
action  5
2: 3
4: 33
5: 116
12: 17
13: 8
14: 13
15: 30
19: 2
****************
action  15
2: 4
4: 4
5: 10
12: 30
13: 2
14: 61
15: 128
19: 1
****************
action  2
1: 1
2: 148
5: 11
8: 2
9: 4
12: 58
13: 11
14: 14
15: 5
16: 1
19: 4
****************
action  12
1: 2
2: 30
3: 1
5: 4
8: 2
9: 2
12: 161
13: 13
14: 13
15: 18
18: 4
19: 14
****************
action  9
2: 4
8: 1
9: 8
19: 3
****************
action  19
2: 1
3: 1
9: 1
12: 3
18: 1
19: 8
****************
action  3
2: 7
3: 26
4: 1
5: 1
6: 1
11: 1
12: 10
13: 5
14: 9
15: 1
19: 1
****************
action  6
3: 1
4: 1
5: 1
6: 10
12: 2
13: 3
14: 2
****************
action  16
12: 2
13: 2
14: 1
15: 1
16: 16
****************
action  1
1: 1
11: 1
****************
action  11
11: 2
****************
action  0
4: 1
12:

In [125]:
bad_count=0
total=0
for k in new_bad_actions:
    print('action ',k)
    x = np.array(new_bad_actions[k])
    unique, counts = np.unique(x, return_counts=True)
    for i in range(len(unique)):
        total+=counts[i]
        if unique[i]!=k:
            bad_count+=counts[i]
        else:
            print("skip ", i)
        print(f'{unique[i]}: {counts[i]}')
    print('****************')

action  13
4: 3
5: 1
skip  2
13: 41
14: 7
15: 5
16: 2
18: 20
****************
action  4
2: 8
skip  1
4: 85
5: 41
12: 2
13: 5
14: 38
15: 13
18: 5
19: 1
****************
action  14
2: 6
3: 3
4: 22
5: 4
12: 1
13: 7
skip  6
14: 109
15: 62
16: 1
18: 19
19: 1
****************
action  5
2: 3
4: 33
skip  2
5: 116
12: 2
13: 8
14: 13
15: 30
18: 15
19: 2
****************
action  15
2: 4
4: 4
5: 10
13: 2
14: 61
skip  5
15: 128
18: 30
19: 1
****************
action  2
1: 1
skip  1
2: 148
5: 11
8: 2
9: 4
12: 15
13: 11
14: 14
15: 5
16: 1
18: 43
19: 4
****************
action  12
1: 2
2: 30
3: 1
5: 4
8: 2
9: 2
skip  6
12: 139
13: 13
14: 13
15: 18
18: 26
19: 14
****************
action  9
2: 4
8: 1
skip  2
9: 8
19: 3
****************
action  19
2: 1
3: 1
9: 1
18: 4
skip  4
19: 8
****************
action  3
2: 7
skip  1
3: 26
4: 1
5: 1
6: 1
11: 1
12: 3
13: 5
14: 9
15: 1
18: 7
19: 1
****************
action  6
3: 1
4: 1
5: 1
skip  3
6: 10
13: 3
14: 2
18: 2
****************
action  16
13: 2
14: 1
15: 1
skip  3

In [101]:
bad_count/total

0.49874937468734365

In [100]:
total

1999

In [97]:
action_stat_correct

897

In [ ]:
total=1
for k in new_bad_actions:
    

In [113]:
for k in bad_empls:
    print('empl ',k)
    x = np.array(bad_empls[k])
    unique, counts = np.unique(x, return_counts=True)
    for i in range(len(unique)):
        print(f'{unique[i]}: {counts[i]}')
    print('****************')
    

empl  6
0: 64
1: 6
2: 2
3: 31
4: 8
5: 12
6: 39
10: 1
11: 5
****************
empl  1
0: 5
1: 1
2: 1
3: 5
6: 12
****************
empl  2
0: 45
1: 6
2: 14
3: 35
4: 6
5: 2
6: 10
10: 3
11: 1
****************
empl  0
0: 296
1: 3
2: 3
3: 83
4: 10
5: 27
6: 37
10: 12
11: 24
****************
empl  4
0: 25
3: 8
4: 2
5: 2
6: 6
10: 4
11: 1
****************
empl  11
0: 116
2: 2
3: 25
4: 4
5: 18
6: 19
10: 6
11: 283
****************
empl  10
0: 65
1: 7
3: 57
5: 27
6: 13
8: 4
10: 43
11: 4
****************
empl  3
0: 44
1: 1
2: 2
3: 108
4: 1
5: 13
6: 5
8: 1
10: 13
11: 2
****************
empl  5
0: 83
1: 5
2: 2
3: 31
4: 2
5: 72
6: 11
8: 1
10: 19
11: 9
****************
empl  8
0: 6
3: 4
5: 3
6: 3
8: 4
10: 3
11: 1
****************


In [37]:
act

13

In [39]:
empl

6

In [ ]:
for ind in test_df.index:
    id=int(test_df['id'][ind])
    act=int(test_df['action'][ind])
    if act not in [4,5,14,15]:
        continue
        
    if id in result_actions:
        if act==result_actions[id]:
            action_stat_correct+=1
        
        if act not in bad_actions:
            bad_actions[act]=[]
        bad_actions[act].append(result_actions[id])
    if id in result_employee:
        if empl==result_employee[id]:
            empl_stat_correct+=1
        if empl not in bad_empls:
            bad_empls[empl]=[]
        bad_empls[empl].append(result_employee[id])

In [114]:
test_df.head()

,id,date,employee,action,date_orig
1266,2522,2022-05-26 08:04:49,6,13.0,2022-05-26 08:04:49
1267,2523,2022-05-26 08:04:54,6,4.0,2022-05-26 08:04:54
1268,2524,2022-05-26 08:07:00,6,14.0,2022-05-26 08:07:00
1269,2525,2022-05-26 08:07:11,6,5.0,2022-05-26 08:07:11
1270,2526,2022-05-26 08:15:16,1,15.0,2022-05-26 08:15:16


In [117]:
test_df[test_df['employee']==6].head(50)

,id,date,employee,action,date_orig
1266,2522,2022-05-26 08:04:49,6,13.0,2022-05-26 08:04:49
1267,2523,2022-05-26 08:04:54,6,4.0,2022-05-26 08:04:54
1268,2524,2022-05-26 08:07:00,6,14.0,2022-05-26 08:07:00
1269,2525,2022-05-26 08:07:11,6,5.0,2022-05-26 08:07:11
1286,2542,2022-05-26 08:42:33,6,15.0,2022-05-26 08:42:33
1287,2543,2022-05-26 08:42:36,6,4.0,2022-05-26 08:42:36
1290,2546,2022-05-26 08:46:17,6,14.0,2022-05-26 08:46:17
1291,2547,2022-05-26 08:46:19,6,5.0,2022-05-26 08:46:19
1292,2548,2022-05-26 09:02:36,6,15.0,2022-05-26 09:02:36
1293,2549,2022-05-26 09:02:38,6,4.0,2022-05-26 09:02:38
